In [1]:
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
username = ""  # your username

with open(f"/home/{username}/ttmp/PBSCR/baselines/9_class_dataset.pkl", "rb") as f:
    (
        x_train9,
        y_train9,
        x_valid9,
        y_valid9,
        x_test9,
        y_test9,
        m_train9,
        m_valid9,
        m_test9,
    ) = pickle.load(f)

with open(f"/home/{username}/ttmp/PBSCR/baselines/100_class_dataset.pkl", "rb") as f:
    (
        x_train100,
        y_train100,
        x_valid100,
        y_valid100,
        x_test100,
        y_test100,
        m_train100,
        m_valid100,
        m_test100,
    ) = pickle.load(f)

composers9 = np.unique(y_train9)
composers100 = np.unique(y_train100)

In [3]:
def get_indices(y_train, composers, N=10):
    composer_pieces = {
        composer: np.argwhere(y_train == composer).flatten() for composer in composers
    }
    indices = np.concatenate(
        [
            np.random.choice(piece_indices, size=(N,), replace=False)
            for piece_indices in composer_pieces.values()
        ]
    )

    return indices

In [4]:
import pandas as pd


# Helper funcs
def expand_repeat(arr, repeats, axis=0):
    return np.repeat(np.expand_dims(arr, axis=axis), repeats, axis=axis)


def fit_and_predict(
    train_features, test_features, indices, y_train, composers, k=3
):  # Function to take x_few and y_few and make guesses about all the x_test samples
    avg_euclideans = np.zeros((len(test_features), len(composers)))
    few_features = train_features[indices]  # L x d (where L is N*#composers)
    batch_size = 128

    for i, composer in enumerate(composers):
        composer_points = np.argwhere(y_train[indices] == composer).flatten()
        euclideans = np.zeros((len(test_features), len(composer_points)))

        for batch in range(
            0, int(batch_size * np.ceil(len(test_features) / batch_size)), batch_size
        ):
            test_batch = test_features[batch : batch + batch_size]
            euclideans[batch : batch + batch_size] = np.sqrt(
                np.sum(
                    np.square(
                        expand_repeat(few_features[composer_points], len(test_batch), 0)
                        - expand_repeat(test_batch, len(composer_points), 1)
                    ),
                    axis=-1,
                )
            )

        avg_euclideans[:, i] = np.mean(np.sort(euclideans, axis=-1)[..., :k], axis=-1)

    ranks = np.argsort(avg_euclideans, axis=-1)

    return ranks

In [5]:
def top_x_acc(y_true, y_pred, x):
    y_true = torch.Tensor(y_true)
    y_pred = torch.Tensor(y_pred)

    top_x = y_pred[..., :x]
    return (
        (top_x == torch.repeat_interleave(y_true.unsqueeze(-1), x, axis=-1))
        .float()
        .sum(-1)
        .mean()
        .item()
    )


def mean_recip_rank(y_true, y_pred):
    y_true = torch.Tensor(y_true)
    y_pred = torch.Tensor(y_pred)

    # starts with worst at 0 but we want best at 1 so
    true_ranks = (
        y_pred
        == torch.repeat_interleave(y_true.unsqueeze(-1), y_pred.shape[-1], axis=-1)
    ).float().argmax(-1) + 1

    return (1 / true_ranks).mean().item()

In [7]:
from tqdm.autonotebook import tqdm

models = ["gpt2", "roberta", "random"]
T = 30
Ns = [1]  # , 10, 100]
datasets = {
    "9": (y_train9, y_test9, composers9),
    # "100":(y_train100, y_test100, composers100)
}

results = {}

for classs, (y_train, y_test, composers) in datasets.items():
    print(f"{classs} class dataset:")

    results[classs] = {}

    for N in Ns:
        results[classs][N] = {}
        print()
        print(f"N: {N}")
        print()

        for model in models:
            train_features, test_features = (
                np.load(f"fewshot_vecs/{model}_train{classs}.npy"),
                np.load(f"fewshot_vecs/{model}_test{classs}.npy"),
            )

            print()
            print(f"Model: {model}")

            top1s = []
            top5s = []
            mrrs = []
            top10s = []

            for _ in tqdm(range(T)):
                indices = get_indices(y_train, composers, N=N)

                y_pred = fit_and_predict(
                    train_features, test_features, indices, y_train, composers, k=3
                )
                y_true = np.array([list(composers).index(i) for i in y_test])

                top1s.append(top_x_acc(y_true, y_pred, 1))
                top5s.append(top_x_acc(y_true, y_pred, 5))
                mrrs.append(mean_recip_rank(y_true, y_pred))

                if classs == "100":
                    top10s.append(top_x_acc(y_true, y_pred, 10))

            results[classs][N][model] = {
                "top1": (np.mean(top1s), np.std(top1s)),
                "top5": (np.mean(top5s), np.std(top5s)),
                "top10": (np.mean(top10s), np.std(top10s)) if top10s else None,
                "mrr": (np.mean(mrrs), np.std(mrrs)),
            }
            print(results[classs][N][model])

9 Way dataset:

N: 1


Model: gpt2


  0%|          | 0/30 [00:00<?, ?it/s]

{'top1': (0.15403409401575724, 0.021760802331783652), 'top5': (0.6186351299285888, 0.03215427742307966), 'top10': None, 'mrr': (0.3595596253871918, 0.020848075841654427)}

Model: roberta


  0%|          | 0/30 [00:00<?, ?it/s]

{'top1': (0.12643678163488706, 0.008172099175515249), 'top5': (0.5828918894131978, 0.014910360959397287), 'top10': None, 'mrr': (0.33158112366994225, 0.008192236926892289)}

Model: random


  0%|          | 0/30 [00:00<?, ?it/s]

{'top1': (0.11172191674510638, 0.004455461120023086), 'top5': (0.5563496053218842, 0.005252118659607039), 'top10': None, 'mrr': (0.31513874530792235, 0.0034284402313440932)}


In [7]:
with open("results.json", "w") as f:
    json.dump(results, f)

In [8]:
import json

In [9]:
with open("results.json", "r") as f:
    results = json.load(f)